In [1]:
import numpy as np
from keras.models import Sequential
from keras.layers import Dense
from keras.utils import np_utils
from keras.models import load_model
from keras import backend as K
import tensorflow as tf
from scipy.misc import imresize
import matplotlib.pyplot as plt
%matplotlib inline

# fix random seed for reproducibility
np.random.seed(7)


Using Theano backend.
WARNING (theano.sandbox.cuda): The cuda backend is deprecated and will be removed in the next release (v0.10).  Please switch to the gpuarray backend. You can get more information about how to switch at this URL:
 https://github.com/Theano/Theano/wiki/Converting-to-the-new-gpu-back-end%28gpuarray%29

Using gpu device 1: GeForce GTX TITAN X (CNMeM is enabled with initial size: 50.0% of memory, cuDNN 5103)


# Load data 

In [2]:
from keras.datasets import mnist
(Xtrain, Ytrain), (Xtest, Ytest) = mnist.load_data()

(ntrain, xdim, ydim) = Xtrain.shape
ntest = Xtest.shape[0]

# split train data in two
X_pr = Xtrain[30000:60000, :, :]
Y_pr = Ytrain[30000:60000]

Xtrain = Xtrain[0:30000, :, :];
Ytrain = Ytrain[0:30000]

# downsample
factor = 1
if factor<1:
    Xtrain_down = np.ones((Xtrain.shape[0], int(xdim*factor), int(ydim*factor)))
    for i in range(Xtrain.shape[0]):
        Xtrain_down[i, :, :] = imresize(Xtrain[i,:,:], factor)

    Xtest_down = np.ones((Xtest.shape[0], int(xdim*factor), int(ydim*factor)))
    for i in range(Xtest.shape[0]):
        Xtest_down[i,:,:] = imresize(Xtest[i,:,:], factor)

    X_pr_down = np.ones((X_pr.shape[0], int(xdim*factor), int(ydim*factor)))
    for i in range(X_pr.shape[0]):
        X_pr_down[i,:,:] = imresize(X_pr[i,:,:], factor)
else:
    Xtrain_down = Xtrain
    Xtest_down = Xtest
    X_pr_down = X_pr
    
# VECTORIZE IMAGES
Xtrain_down = Xtrain_down.reshape(Xtrain_down.shape[0], int(xdim*factor)**2).astype('float32') / 255
Xtest_down = Xtest_down.reshape(ntest, int(xdim*factor)**2).astype('float32') / 255
X_pr_down = X_pr_down.reshape(X_pr_down.shape[0], int(xdim*factor)**2).astype('float32') / 255

# CATEGORICAL OUTPUTS
Ytrain_cat = np_utils.to_categorical(Ytrain, 10)
Ytest_cat = np_utils.to_categorical(Ytest, 10)


# TRAIN A FULLY-CONNECTED NN WITH 4 ReLU HIDDEN LAYERS 

In [3]:
# create model
model = Sequential()
model.add(Dense(128, input_shape=(int(xdim*factor)**2,), activation='relu'))
model.add(Dense(128, activation='relu'))
model.add(Dense(128, activation='relu'))
model.add(Dense(128, activation='relu'))
model.add(Dense(10, activation='softmax'))

# Compile model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
# model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
 
model.fit(Xtrain_down, Ytrain_cat, nb_epoch=50, batch_size=16, verbose=2)

# evaluate the model
scores = model.evaluate(Xtest_down, Ytest_cat, verbose=0)
print("\n%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))

# Load NN and get output values
# from keras import backend as K
# from keras.models import load_model

# Save the model
#model.save('nonlinear_nn.h5')

# GET THE OUTPUT OF EACH LAYER AFTER TRAINING
inp = model.input                                           # input placeholder
outputs = [layer.output for layer in model.layers]          # all layer outputs
functor = K.function([inp]+ [K.learning_phase()], outputs ) # evaluation function
layer_outs = functor([X_pr_down, 1.])                       # compute on 2nd training set
layer_outs_test = functor([Xtest_down, 1.])                 # compute on test set
#print(layer_outs)



/opt/anaconda/anaconda2/lib/python2.7/site-packages/keras/models.py:826: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  warnings.warn('The `nb_epoch` argument in `fit` '


Epoch 1/50
1s - loss: 0.2993 - acc: 0.9084
Epoch 2/50
1s - loss: 0.1327 - acc: 0.9594
Epoch 3/50
1s - loss: 0.0959 - acc: 0.9716
Epoch 4/50
1s - loss: 0.0774 - acc: 0.9765
Epoch 5/50
1s - loss: 0.0648 - acc: 0.9799
Epoch 6/50
1s - loss: 0.0549 - acc: 0.9836
Epoch 7/50
1s - loss: 0.0470 - acc: 0.9864
Epoch 8/50
1s - loss: 0.0407 - acc: 0.9874
Epoch 9/50
1s - loss: 0.0375 - acc: 0.9894
Epoch 10/50
1s - loss: 0.0312 - acc: 0.9908
Epoch 11/50
1s - loss: 0.0337 - acc: 0.9897
Epoch 12/50
1s - loss: 0.0280 - acc: 0.9923
Epoch 13/50
1s - loss: 0.0284 - acc: 0.9918
Epoch 14/50
1s - loss: 0.0256 - acc: 0.9930
Epoch 15/50
1s - loss: 0.0230 - acc: 0.9928
Epoch 16/50
1s - loss: 0.0248 - acc: 0.9929
Epoch 17/50
1s - loss: 0.0205 - acc: 0.9942
Epoch 18/50
1s - loss: 0.0193 - acc: 0.9943
Epoch 19/50
1s - loss: 0.0230 - acc: 0.9937
Epoch 20/50
1s - loss: 0.0183 - acc: 0.9952
Epoch 21/50
1s - loss: 0.0243 - acc: 0.9939
Epoch 22/50
1s - loss: 0.0193 - acc: 0.9948
Epoch 23/50
1s - loss: 0.0168 - acc: 0.99

# Save Layer Outputs

In [4]:
# import pickle
# with open('results/NN_layer_outputs.dat','wb') as f:
#     pickle.dump([layer_outs, layer_outs_test], f)
# f.close()

In [6]:
layer_outs

[array([[ 0.        ,  0.        ,  0.        , ...,  0.        ,
          0.        ,  0.        ],
        [ 0.        ,  3.16805243,  0.        , ...,  1.25666952,
          0.        ,  2.58325386],
        [ 0.        ,  0.        ,  4.39460564, ...,  0.        ,
          0.        ,  0.        ],
        ..., 
        [ 0.        ,  0.        ,  4.16263247, ...,  0.        ,
          0.24436386,  0.        ],
        [ 0.        ,  0.        ,  0.        , ...,  0.42510724,
          4.08305883,  0.        ],
        [ 0.        ,  0.        ,  0.        , ...,  0.        ,
          0.        ,  0.        ]], dtype=float32),
 array([[  0.        ,   0.        ,   0.        , ...,   0.38174966,
           0.        ,   0.        ],
        [  0.        ,   0.        ,  14.20276451, ...,   0.        ,
           7.18004131,   0.        ],
        [  0.        ,   0.        ,   0.        , ...,   0.        ,
           0.20144165,   0.        ],
        ..., 
        [  0.      